In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mne
import os
import sys
from mne.datasets import eegbci
import glob
from IPython.display import clear_output
import numpy as np
import torch
from torch import nn
import torch.optim as optim

from mne.datasets import eegbci
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from scipy import signal

import torch
import torch.cuda as cuda
import torch.nn as nn
from torch.autograd import Variable
import math
import wandb
import os
from common import train
from common import create_dataloader
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import wandb
wandb.login()
os.environ['http_proxy'] = "http://192.41.170.23:3128" 
os.environ['https_proxy'] = "http://192.41.170.23:3128" 

In [ ]:
torch.cuda.is_available()
torch.cuda.get_device_name(0)

In [ ]:
from common import EEG


In [ ]:

# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, out_size):
        super().__init__()
        #using sequential helps bind multiple operations together
        self.layer1 = nn.Sequential(
            #padding = (kernel_size - 1) / 2 = 2
            nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, stride=1, padding=2),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv1d(out_channels, out_channels*2, kernel_size=kernel_size, stride=1, padding=2),
            nn.BatchNorm1d(out_channels*2),
            nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv1d(out_channels*2, out_channels, kernel_size=kernel_size, stride=1, padding=2),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(),
        )
        
        
        self.fc = nn.Linear(linear_shape  , out_size)
        #self.fc = nn.Linear(80064 , 2)
        
        self.drop_out = nn.Dropout(0.5)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.drop_out(out)
        out = self.layer2(out)
        out = self.drop_out(out)
        out = self.layer3(out)
        #out = self.drop_out(out)
        #out = self.layer4(out)
        #out = self.drop_out(out)

        out = out.reshape(out.size(0), -1)   #can also use .view()
        out = self.fc(out)
        return out

In [ ]:
subjects_all = list(range(1, 110))
filter_num =[38,88,89,92,100,104]
subjects = [subject for subject in subjects_all if subject not in filter_num]


#print(subjects)
#[20,31,47,48,49,54,55,59,62]# 

in_channels  = 2 #C3, C4
out_channels = 64  #five is logical because we have freq= 8, 9, 10, 11, 12 that we want to capture
out_size     = 2  #left or right
kernel_size  = 5




In [ ]:
##PACK
for i in range(3):
    if i == 0:
        n_type = "physionet_EX"
    if i == 1:
        n_type = "physionet_MI"
    if i == 2:
        n_type = "physionet_ALL"

    if n_type == "physionet_EX":
        runs = [3,7,11]
    if n_type == "physionet_MI":
        runs = [4,8,12]
    if n_type == "physionet_ALL":
        runs = [3,7,11,4,8,12]
    
    print("############## {} ########".format(subjects))
    path = 'physionet.org/files/eegmmidb/1.0.0'
    base_url = 'https://physionet.org/files/eegmmidb/'
    eeg = EEG(path, base_url, subjects, runs)
    raw=eeg.data_to_raw()
    print("Raw done")
    
    raw=raw.notch_filter([50])
    raw=raw.filter( 8,14, method='fir', verbose=20)
    raw=raw.pick_channels(['C3', 'C4'])
    eeg.create_epochs()
    X, y = eeg.get_X_y()

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y)

    print('train size',X_train.shape, y_train.shape)
    print('Test size',X_test.shape, y_test.shape)
    batch_size = X_train.shape[2]
    train_loader = create_dataloader(X_train, y_train, batch_size=batch_size)
    test_loader = create_dataloader(X_test, y_test, batch_size=batch_size)
    num_step =math.ceil(len(train_loader.dataset) / batch_size)

    x_train = torch.FloatTensor(X_train)
    fake_eeg_data = torch.randn_like(x_train) #(batch_size, channel, length)

        #using sequential helps bind multiple operations together
    layer1 = nn.Sequential(
                #padding = (kernel_size - 1) / 2 = 2
                nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, stride=1, padding=2),
                nn.BatchNorm1d(out_channels),
                nn.ReLU()
                # nn.MaxPool2d(kernel_size=2, stride=2)
            )
    layer2 = nn.Sequential(
                nn.Conv1d(out_channels, out_channels*2, kernel_size=kernel_size, stride=1, padding=2),
                nn.BatchNorm1d(out_channels*2),
                nn.ReLU()
                # nn.MaxPool2d(kernel_size=2, stride=2)
            )
    layer3 = nn.Sequential(
                nn.Conv1d(out_channels*2, out_channels, kernel_size=kernel_size, stride=1, padding=2),
                nn.BatchNorm1d(out_channels),
                nn.ReLU()
            )


    out = layer1(fake_eeg_data)
    out = layer2(out)
    out = layer3(out)
    out = out.reshape(out.size(0), -1)
    linear_shape = out.shape[1]
    print(linear_shape)
    fc = nn.Linear(linear_shape, out_size)
    out = fc(out)

    wand = wandb.init(

          # Set the project where this run will be logged
          project="Motor-Imagery", 
          # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
          name="CNN_3layer_2class_physionet_{}_newlr".format(n_type), 
          # Track hyperparameters and run metadata
          config={
          "learning_rate": 0.00001,
          "architecture": "CNN",
          "dataset": "S{}".format(subjects),
          "epochs": 1000,
          "weightname":"CNN_3layer_2class_physionet_{}_newlr".format(n_type),
          "num_step_per_epoch" : num_step, 

          }
        )
    wandb.login()
    config = wand.config
    net = ConvNet(in_channels, out_channels, kernel_size, out_size).cuda(0)
    optimizer = optim.Adam(net.parameters(), lr=config.learning_rate)
    criterion = nn.CrossEntropyLoss()


    train_loss,valid_loss,train_accuracy,valid_accuracy =train(
        model = net,
        gpu_num = 0,
        train_loader = train_loader,
        test_loader = test_loader,
        optimizer = optimizer  ,
        criterion = criterion ,
        wand = wand
             )


    wandb.alert(
                title='Finish',
                text=f'Finishing training',
            )

In [ ]:
for subject in subjects:
    print("############## {} ########".format(subject))
    
    path = 'physionet.org/files/eegmmidb/1.0.0'
    base_url = 'https://physionet.org/files/eegmmidb/'
    n_type = "physionet_MI"
    
    # subjects = [1]
    runs = [3, 4, 7, 8, 11, 12]
    #runs = [4, 8, 12]
    subjects = [subject]#[i for i in range(1,40)]
    #subjects = [1]
    # runs = [6,10,14]

    eeg = EEG(path, base_url, subjects, runs)
    raw=eeg.data_to_raw()
    print("Raw done")
    # apply filter
    
    raw=raw.notch_filter([50])
    raw=raw.filter( 8,14, method='fir', verbose=20)
    raw=raw.pick_channels(['C3', 'C4'])
    eeg.create_epochs()
    
    print(raw.ch_names)
    X, y = eeg.get_X_y()
    print(y)
    #print(X)
    #X2 = X[:, 8:9, :] 
    #X3= X[:, 12:13, :]
    #X = np.concatenate((X2,X3), axis=1)
    #y=y-1
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y)

    print('train size',X_train.shape, y_train.shape)
    print('Test size',X_test.shape, y_test.shape)
    batch_size = X_train.shape[2]
    train_loader = create_dataloader(X_train, y_train, batch_size=batch_size)
    test_loader = create_dataloader(X_test, y_test, batch_size=batch_size)
    num_step =math.ceil(len(train_loader.dataset) / batch_size)
    
    x_train_fake = torch.FloatTensor(X_train)
    fake_eeg_data = torch.randn_like(x_train_fake) #(batch_size, channel, length)
    X_train = torch.FloatTensor(X_train)
            #using sequential helps bind multiple operations together
    layer1 = nn.Sequential(
                #padding = (kernel_size - 1) / 2 = 2
                nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, stride=1, padding=2),
                nn.BatchNorm1d(out_channels),
                nn.ReLU()
                # nn.MaxPool2d(kernel_size=2, stride=2)
            )
    layer2 = nn.Sequential(
                nn.Conv1d(out_channels, out_channels*2, kernel_size=kernel_size, stride=1, padding=2),
                nn.BatchNorm1d(out_channels*2),
                nn.ReLU()
                # nn.MaxPool2d(kernel_size=2, stride=2)
            )
    layer3 = nn.Sequential(
                nn.Conv1d(out_channels*2, out_channels, kernel_size=kernel_size, stride=1, padding=2),
                nn.BatchNorm1d(out_channels),
                nn.ReLU()
            )

    out = layer1(fake_eeg_data)
    out = layer2(out)
    out = layer3(out)
    out = out.reshape(out.size(0), -1)
    linear_shape = out.shape[1]
    linear_shape
    fc = nn.Linear(linear_shape, out_size)
    out = fc(out)
    out.shape
    
    wand = wandb.init(

          # Set the project where this run will be logged
          project="Motor-Imagery", 
          # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
          name="CNN_3layer_2class_S{0}_{1}".format(subject,n_type), 
          # Track hyperparameters and run metadata
          config={
          "learning_rate": 0.000000001,
          "architecture": "CNN",
          "dataset": "S{}".format(subjects[0]),
          "epochs": 300000,
          "weightname":"CNN_3layer_2class_S{0}_{1}".format(subject,n_type),
          "num_step_per_epoch" : num_step, 

          }
        )
    wandb.login()
    config = wand.config
    net = ConvNet(in_channels, out_channels, kernel_size, out_size).cuda(0)
    optimizer = optim.Adam(net.parameters(), lr=config.learning_rate)
    criterion = nn.CrossEntropyLoss()


    train_loss,valid_loss,train_accuracy,valid_accuracy =train(
        model = net,
        gpu_num = 0,
        train_loader = train_loader,
        test_loader = test_loader,
        optimizer = optimizer  ,
        criterion = criterion ,
        wand = wand
             )


    wandb.alert(
                title='Finish',
                text=f'Finishing training',
            )

In [ ]:
from common import EEG_fif
from common import create_dataloader

In [ ]:
subjects = [20,31,48,50,54,59,62]
print("subjects number : {} ".format(subjects))

#[20,31,47,48,49,54,55,59,62]# list(range(36, 63))

in_channels  = 2 #C3, C4
out_channels = 64  #five is logical because we have freq= 8, 9, 10, 11, 12 that we want to capture
out_size     = 2  #left or right
kernel_size  = 5



path = ''
base_url = 'https://physionet.org/files/eegmmidb/'



In [ ]:
for subject in subjects:
    for i in range(3):
        if i == 0:
            n_type = "physionet_EX"
            part_weight = "CNN_3layer_2class_physionet_physionet_EX_newlr/0.6923_CNN_3layer_2class_physionet_physionet_EX_newlr_0.6923_50.0000"
            runs = [3,5,7,9]
            if subject == 31
        if i == 1:
            n_type = "physionet_MI"
            part_weight = "CNN_3layer_2class_physionet_physionet_MI_newlr/0.6904_CNN_3layer_2class_physionet_physionet_MI_newlr_0.6904_53.5018"
            runs = [4,6,8,10]
        if i == 2:
            n_type = "physionet_ALL"
            part_weight = "CNN_3layer_2class_physionet_physionet_ALL_newlr/0.6929_CNN_3layer_2class_physionet_physionet_ALL_newlr_0.6929_50.2349"
            runs = [3,5,7,9,4,6,8,10]

            
        print("############## {} ########".format(subject))
        
        eeg = EEG_fif(path, base_url, subjects, runs)
        raw=eeg.data_to_raw()
        print("Raw done")
        raw=raw.notch_filter([50])
        raw=raw.filter( 8,14, method='fir', verbose=20)
        raw.resample(160)
        print("Filter done")
        raw.pick_channels(['C3', 'C4', 'STIM MARKERS'])
        epochs=eeg.epochs_visu(raw,tmin=0,tmax=7,baseline=(0,1))
        X, y = eeg.get_X_y(epochs)
        X=X[:,:,int(2*160):int(1211-(250*1))]
        y=y-1
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y)


        print('train size',X_train.shape, y_train.shape)
        print('Test size',X_test.shape, y_test.shape)
        batch_size = X_train.shape[2]
        train_loader = create_dataloader(X_train, y_train, batch_size=batch_size)
        test_loader = create_dataloader(X_test, y_test, batch_size=batch_size)
        num_step =math.ceil(len(train_loader.dataset) / batch_size)

        x_train = torch.FloatTensor(X_train)
        fake_eeg_data = torch.randn_like(x_train) #(batch_size, channel, length)

            #using sequential helps bind multiple operations together
        layer1 = nn.Sequential(
                    #padding = (kernel_size - 1) / 2 = 2
                    nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, stride=1, padding=2),
                    nn.BatchNorm1d(out_channels),
                    nn.ReLU()
                    # nn.MaxPool2d(kernel_size=2, stride=2)
                )
        layer2 = nn.Sequential(
                    nn.Conv1d(out_channels, out_channels*2, kernel_size=kernel_size, stride=1, padding=2),
                    nn.BatchNorm1d(out_channels*2),
                    nn.ReLU()
                    # nn.MaxPool2d(kernel_size=2, stride=2)
                )
        layer3 = nn.Sequential(
                    nn.Conv1d(out_channels*2, out_channels, kernel_size=kernel_size, stride=1, padding=2),
                    nn.BatchNorm1d(out_channels),
                    nn.ReLU()
                )


        out = layer1(fake_eeg_data)
        out = layer2(out)
        out = layer3(out)
        out = out.reshape(out.size(0), -1)
        linear_shape = out.shape[1]
        fc = nn.Linear(linear_shape, out_size)
        out = fc(out)

        wand = wandb.init(

              # Set the project where this run will be logged
              project="Motor-Imagery", 
              # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
              name="CNN_3layer_2class_physionet_S{0}_{1}".format(subject,n_type), 
              # Track hyperparameters and run metadata
              config={
              "learning_rate": 0.00001,
              "architecture": "CNN",
              "dataset": "S{0}".format(subject),
              "epochs": 100000,
              "weightname":"CNN_3layer_2class_physionet_S{0}_{1}".format(subject,n_type),
              "num_step_per_epoch" : num_step, 

              }
            )
        wandb.login()
        config = wand.config
        net = ConvNet(in_channels, out_channels, kernel_size, out_size).cuda(0)
        net.load_state_dict(torch.load(part_weight))
        for name,param in net.named_parameters():
            if param.requires_grad and 'layer1' in name:
                param.requires_grad = False
            if param.requires_grad and 'layer2' in name:
                param.requires_grad = False
        
        optimizer = optim.Adam(net.parameters(), lr=config.learning_rate)
        criterion = nn.CrossEntropyLoss()


        train_loss,valid_loss,train_accuracy,valid_accuracy =train(
            model = net,
            gpu_num = 0,
            train_loader = train_loader,
            test_loader = test_loader,
            optimizer = optimizer  ,
            criterion = criterion ,
            wand = wand
                 )


        wandb.alert(
                    title='Finish',
                    text=f'Finishing training',
                )